# Introduction
This notebook goes through multi-processing of NLP for aiming to analyse the dependency of David Bowie with the Media's

Specifically, we'll be walking through:

 - **Getting the data** - in this case, we'll be get the data from a dataset in .csv
 - **Cleaning the data** - we will walk through popular text pre-processing techniques
 - **Organizing the data** - we will organize the cleaned data into a way that is easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

  - Corpus - a collection of text
  - Document-Term Matrix  word counts in matrix format


In each formats, we'll divide in 3 category:

  - The press - the magazine from the text
  - The year of publish - The year of the interview
  - The album - Each interview is linked with an album

### Problem Statement

We would like to know if trough the decade, the review about Bowie evolute.

# Get the data

Luckily, we already have a raw csv, with all our data ready. I have to import the csv, and transform them in dataframe
for each categories.

As we decided before we have to divide the dataset in 3 high-level category.

In [5]:
import pandas as pd
import pickle
import numpy as np

file = r'data/bowie_txt_analysis.csv'
df = pd.read_csv(file)

# Pickle all the data
all_text = open('./data/pickle/all_text.pkl', 'wb')
pickle.dump(df, all_text)

# Create separate df for each category
df_press = df[['press', 'texte', 'date']]
df_album = df[['album', 'texte', 'date']]

def format_date(date):
    new_date = int(date[-2:])
    if new_date > 20:
        new_date = f'19{new_date}'
    elif new_date == 2 or new_date == 3:
        new_date = f'200{new_date}'
    else:
        new_date = f'20{new_date}'
    return int(new_date)

format_date_round = lambda x: format_date(x)
df_press.date = df_press.date.apply(format_date_round)
df_press = df_press.groupby(['date','texte', 'press']).size().reset_index()

df_press_70 = df_press[df_press['date'].between(1970, 1979)]
df_press_70 = df_press_70.groupby(['press'])['texte'].apply(' '.join).reset_index()
df_press_80 = df_press[df_press['date'].between(1980, 1989)]
df_press_80 = df_press_80.groupby(['press'])['texte'].apply(' '.join).reset_index()
df_press_90 = df_press[df_press['date'].between(1990, 1999)]
df_press_90 = df_press_90.groupby(['press'])['texte'].apply(' '.join).reset_index()
df_press_00 = df_press[df_press['date'].between(2000, 2009)]
df_press_00 = df_press_00.groupby(['press'])['texte'].apply(' '.join).reset_index()
df_press_10 = df_press[df_press['date'].between(2010, 2019)]


ModuleNotFoundError: No module named 'pandas'

# Cleaning the data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text**:

- Make text all lower case
- Remove punctuation
- Remove numerical values
- Remove common non-sensical text (/n)
- Tokenize text
- Remove stop words


**More data cleaning steps after tokenization**:
- Stemming / lemmatization
- Parts of speech tagging
- Create bi-grams or tri-grams
- Deal with typos
- And more...


In [27]:
import re
import string

def clean_text_round1(text):
    """Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers."""
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = text.lstrip(' ')
    return text

def clean_text_round2(text):
    """Get rid of some additional punctuation and non-sensical text that was missed the first time around."""
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

def remove_redundant_word(text):
    stopwords = ['bowie', 'like', 'bowies', 'just', 'album', 'david', 'time', 'new']
    querywords = text.split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    return result

round1 = lambda x: clean_text_round1(x)
round2 = lambda x: clean_text_round2(x)
round_stop = lambda x: remove_redundant_word(x)

df_press_70['texte'] = pd.DataFrame(df_press_70.texte.apply(round1))
df_press_70['texte'] = pd.DataFrame(df_press_70.texte.apply(round2))
df_press_70['texte'] = pd.DataFrame(df_press_70.texte.apply(round_stop))

df_press_80['texte'] = pd.DataFrame(df_press_80.texte.apply(round1))
df_press_80['texte'] = pd.DataFrame(df_press_80.texte.apply(round2))
df_press_80['texte'] = pd.DataFrame(df_press_80.texte.apply(round_stop))

df_press_90['texte'] = pd.DataFrame(df_press_90.texte.apply(round1))
df_press_90['texte'] = pd.DataFrame(df_press_90.texte.apply(round2))
df_press_90['texte'] = pd.DataFrame(df_press_90.texte.apply(round_stop))

df_press_00['texte'] = pd.DataFrame(df_press_00.texte.apply(round1))
df_press_00['texte'] = pd.DataFrame(df_press_00.texte.apply(round2))
df_press_00['texte'] = pd.DataFrame(df_press_00.texte.apply(round_stop))

# Pickle press df
press_pkl = open('./data/pickle/press.pkl', 'wb')
pickle.dump(df_press, press_pkl)

press_70_pkl = open('./data/pickle/decade/press_70.pkl', 'wb')
pickle.dump(df_press_70, press_70_pkl)

press_80_pkl = open('./data/pickle/decade/press_80.pkl', 'wb')
pickle.dump(df_press_80, press_80_pkl)

press_90_pkl = open('./data/pickle/decade/press_90.pkl', 'wb')
pickle.dump(df_press_90, press_90_pkl)

press_00_pkl = open('./data/pickle/decade/press_00.pkl', 'wb')
pickle.dump(df_press_00, press_00_pkl)

# Organizing The Data
I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:

 - **Corpus** - a collection of text
 - **Document**-Term Matrix - word counts in matrix format

## Document-Term Matrix
For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')

## for the 70's
data_press_70_cv = cv.fit_transform(df_press_70['texte'])
data_press_70_dtm = pd.DataFrame(data_press_70_cv.toarray(), columns=cv.get_feature_names())
data_press_70_dtm.index = df_press_70.press
data_press_70_dtm_pkl = open('./data/pickle/decade/press_70_dtm.pkl', 'wb')
pickle.dump(data_press_70_dtm, data_press_70_dtm_pkl)

## for the 80's
data_press_80_cv = cv.fit_transform(df_press_80['texte'])
data_press_80_dtm = pd.DataFrame(data_press_80_cv.toarray(), columns=cv.get_feature_names())
data_press_80_dtm.index = df_press_80.press
data_press_80_dtm_pkl = open('./data/pickle/decade/press_80_dtm.pkl', 'wb')
pickle.dump(data_press_80_dtm, data_press_80_dtm_pkl)

## for the 90's
data_press_90_cv = cv.fit_transform(df_press_90['texte'])
data_press_90_dtm = pd.DataFrame(data_press_90_cv.toarray(), columns=cv.get_feature_names())
data_press_90_dtm.index = df_press_90.press
data_press_90_dtm_pkl = open('./data/pickle/decade/press_90_dtm.pkl', 'wb')
pickle.dump(data_press_90_dtm, data_press_90_dtm_pkl)

## for the 00's
data_press_00_cv = cv.fit_transform(df_press_00['texte'])
data_press_00_dtm = pd.DataFrame(data_press_00_cv.toarray(), columns=cv.get_feature_names())
data_press_00_dtm.index = df_press_00.press
data_press_00_dtm_pkl = open('./data/pickle/decade/press_00_dtm.pkl', 'wb')
pickle.dump(data_press_90_dtm, data_press_00_dtm_pkl)

# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
import pickle
pickle.dump(cv, open("./data/pickle/cv.pkl", "wb"))
